## **Elaborazione di Immagini Mediche**
### Contest 2021/22 - Segmentazione ghiandola prostatica in immagini MRI

### Script di Training

Rigazio Sofia, Roccaro Lucia, Romano Anastasio, Ruzzante Elena

* Installazione delle librerie necessarie

In [ ]:
# Before running the script, reset the runtime to factory reset (Runtime -> Factory Reset Runtime)
# and then change runtime type to GPU (Runtime -> Change runtime type)

!pip install tensorflow==2.1.0
!pip install keras==2.3.1
!pip install segmentation_models==1.0.1
!pip install h5py==2.10.0 
!pip install plotly==5.3.1



*   Collegamento a Google Drive e import delle librerie necessarie



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import random
import numpy as np
import plotly.express as px

from matplotlib import pyplot as plt
from tqdm import tqdm
from skimage.io import imread
from skimage.transform import resize
from skimage.segmentation import mark_boundaries

from keras.callbacks import CSVLogger
from keras.callbacks import EarlyStopping
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

from segmentation_models import Unet

# aggiunti
from scipy import ndimage

*   Preparazione del dataset

In [ ]:
current_dir = 'drive/MyDrive/Colab Notebooks/EIM/Contest 2021-22/'
dataset_name = 'DATASET_stu'

# Path
TRAIN_IMG_path = os.path.join(current_dir,dataset_name,'train','images')
TRAIN_MASK_path = os.path.join(current_dir,dataset_name,'train','manual')
VAL_IMG_path = os.path.join(current_dir,dataset_name,'val','images')
VAL_MASK_path = os.path.join(current_dir,dataset_name,'val','manual')

In [ ]:
# Caricamento dataset in formato .rar da Google Drive a Colab
!pip install unrar
!unrar x "drive/MyDrive/Colab Notebooks/EIM/Contest 2021-22/DATASET_stu.rar" "drive/MyDrive/Colab Notebooks/EIM/Contest 2021-22/"

* Estrazione della lista di volumi del training e validation set

In [ ]:
train_images = os.listdir(TRAIN_IMG_path)
n_images_train = len(train_images)
val_images = os.listdir(VAL_IMG_path)
n_images_val = len(val_images)

* Lettura di un'immagine e della rispettiva maschera e settaggio dei parametri

In [ ]:
# Lettura di un generico volume e rispettiva maschera manuale
chosen_image_name = train_images[0] # prendiamo la prima immagine
original_volume = imread(os.path.join(TRAIN_IMG_path, chosen_image_name))
mask_manual = imread(os.path.join(TRAIN_MASK_path, chosen_image_name))

# immagini
print(original_volume.shape)
print(original_volume.dtype)
# le immagini hanno dimensione (24,512,512) -> sono composte da 24 slice in grayscale
# il formato delle immagini è uint8

# definiamo quindi i parametri
n_slices = original_volume.shape[0]
IMG_WIDTH = original_volume.shape[2] # la larghezza sono le colonne
IMG_HEIGHT = original_volume.shape[1] # l'altezza sono le righe
NUM_CLASSES = 2 # vogliamo segmentare due classi: prostata e background
IMG_CHANNELS = 3 # Le immagini fornite sono grayscale, per poter utilizzare i pesi preallenati di ImageNet forzeremo le immagini a RGB

# maschere
print(mask_manual.dtype)
print(mask_manual.max())
# il formato delle maschere è uint8
# le maschere hanno valori 0 e 1 -> 0: background, 1: prostata

* Rappresentazione a video del volume di un'immagine e della maschera corrispondente


In [ ]:
# Rappresentazione a video del volume prostatico
fig = px.imshow(original_volume, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()
# Rappresentazione a video della corrispondente maschera binaria
fig = px.imshow(mask_manual, animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()

* Visualizzazione del contorno dell'oggetto su una slice dell'immagine originale

In [ ]:
# Estrazione della decima slice
slice_chosen = 10
slice_10 = original_volume[slice_chosen-1,:,:]
mask_10 = mask_manual[slice_chosen-1,:,:]

# Plot a video dell'immagine originale e della corrispondente segmentazione manuale
fig = plt.figure(figsize=(25,25))
ax1 = fig.add_subplot(1,3,1)
ax1.imshow(slice_10,cmap=plt.cm.gray), ax1.set_title('original image')

ax2 = fig.add_subplot(1,3,2)
ax2.imshow(mask_10,cmap=plt.cm.gray), ax2.set_title('manual segmentation')    

ax3 = fig.add_subplot(1,3,3)
ax3.imshow(mark_boundaries(slice_10,mask_10,color=(1,0,0))), ax3.set_title('image + manual')  

* Definizione di una funzione che effettua il preprocessing delle immagini

In [ ]:
def preprocess(image,mask):  # quando copiato nello script di testing togliere maschera in input e output dalla funzione preprocess!!

  # RIMOZIONE DEL BORDO NERO (SE PRESENTE)
  # rimuoviamo le colonne completamente nere
  idx = np.argwhere(np.all(image == 0, axis=0))
  image = np.delete(image, idx, axis=1)
  mask = np.delete(mask, idx, axis=1)
  # rimuoviamo le righe completamente nere
  idx = np.argwhere(np.all(image == 0, axis=1))
  image = np.delete(image, idx, axis=0)
  mask = np.delete(mask, idx, axis=0)
  # riportiamo alla dimensione originale (solo se ne abbiamo modificato le dimensioni)
  if image.shape != (IMG_HEIGHT,IMG_WIDTH):
    image = resize(image, (IMG_HEIGHT,IMG_WIDTH), preserve_range=True).astype(np.uint8)
    mask = resize(mask, (IMG_HEIGHT,IMG_WIDTH), preserve_range=True).astype(np.float32)

  # filtro gaussiano
  def kernel_gauss(size, sigma):
    v = np.linspace(-(size-1)/2,(size-1)/2,size)
    x, y = np.meshgrid(v,v)
    h = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    h = h/h.sum()
    return h

  image = ndimage.correlate(image,kernel_gauss(3,3))

  return image, mask

* Creazione delle matrici delle immagini e delle maschere

In [ ]:
def create_matrix (list_images,n_images,IMG_path,MASK_path):
  # creazione delle matrici che conterranno le slice
  X = np.zeros((n_images*n_slices,IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS), dtype=np.uint8) # immagini
  Y = np.zeros((n_images*n_slices,IMG_WIDTH,IMG_HEIGHT,NUM_CLASSES), dtype=np.float32) # maschere

  for n, id_ in tqdm(enumerate(list_images), total=len(list_images)):
    # la variabile "n" rappresenta un contatore (0-num_immagini) mentre "id_" 
    # contiene il nome della n-esima immagine

    # Lettura dell'immagine e della maschera -> 24 slice di dimensione 512x512
    image = imread(os.path.join(IMG_path, id_))
    mask = imread(os.path.join(MASK_path, id_))
    
    for i in range(n_slices): # consideriamo una slice alla volta
      img_slice = image[i]
      mask_slice = mask[i]
      
      # Preprocessing
      [img_slice,mask_slice] = preprocess(img_slice,mask_slice)

      # Forziamo l'immagine a RGB impilando 3 layer contenenti gli stessi valori 
      X[n*n_slices+i] = np.stack((img_slice,img_slice,img_slice), axis = 2)

      # Conversione della maschera in dato categorico
      mask_cat = to_categorical(mask_slice, num_classes=NUM_CLASSES, dtype='float32')
      # Inserimento della maschera nella matrice
      Y[n*n_slices+i] = mask_cat

  return(X,Y)

In [ ]:
# Training Set
print('Reading images - Training set')
[X_train,Y_train] = create_matrix(train_images,n_images_train,TRAIN_IMG_path,TRAIN_MASK_path)

# Validation Set
print('\nReading images - Validation set')
[X_val,Y_val] = create_matrix(val_images,n_images_val,VAL_IMG_path,VAL_MASK_path)


* Verifica di corretta creazione delle matrici

In [ ]:
# Training Set
index = random.randint(0,n_images_train-1)
# Rappresentazione a video del volume prostatico
fig = px.imshow(X_train[index*n_slices:(index+1)*n_slices-1], animation_frame=0, labels=dict(animation_frame="slice"))
fig.show()
# Rappresentazione a video della corrispondente maschera binaria
fig = px.imshow(Y_train[index*n_slices:(index+1)*n_slices-1,:,:,1], animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()

In [ ]:
# Validation Set
index = random.randint(0,n_images_val-1)
# Rappresentazione a video del volume prostatico
fig = px.imshow(X_val[index*n_slices:(index+1)*n_slices-1], animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()
# Rappresentazione a video della corrispondente maschera binaria
fig = px.imshow(Y_val[index*n_slices:(index+1)*n_slices-1,:,:,1], animation_frame=0, binary_string=True, labels=dict(animation_frame="slice"))
fig.show()

* Definizione della funzione di Data Augmentation

In [ ]:
# Data augmentation (training set)
image_datagen = ImageDataGenerator(rotation_range = 180,
									width_shift_range = 0.2,
									height_shift_range = 0.2,
									horizontal_flip = True,
									vertical_flip = True,
									fill_mode = 'reflect')

# Data augmentation (validation set) -> non settiamo parametri perché fare data augmentation sul validation set è un errore
val_datagen = ImageDataGenerator()

# Generator
seed = 1
def XYaugmentGenerator(X1, y, seed, batch_size):
	genX1 = image_datagen.flow(X1, y, batch_size=batch_size, seed=seed)
	genX2 = image_datagen.flow(y, X1, batch_size=batch_size, seed=seed)
	while True:
		X1i = genX1.next()
		X2i = genX2.next()
		yield X1i[0], X2i[0]

* Definizione del modello UNET

In [ ]:
BACKBONE = 'resnet34'
model = Unet(backbone_name = BACKBONE,
            input_shape = (IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS),
            encoder_weights = 'imagenet', 
            encoder_freeze = True,
            decoder_block_type = 'transpose',
            classes = NUM_CLASSES,
            activation = 'sigmoid')

# Definizione algoritmo di ottimizzazione e funzione di loss
model.compile('Adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

* Allenamento della rete

In [ ]:
# Allenamento della rete

# Parametri della rete
n_train_samples = n_images_train*n_slices # numero delle immagini di train
n_val_samples = n_images_val*n_slices # numero delle immagini di validation
batch_size = 8
n_epochs = 20

# Checkpoint definition
csv_logger = CSVLogger('./log.out', append=True, separator=';')
earlystopping = EarlyStopping(monitor = 'val_binary_accuracy',verbose = 1, min_delta = 0.01, patience = 4, mode = 'max')
callbacks_list = [csv_logger, earlystopping]

# Train model
results = model.fit_generator(XYaugmentGenerator(X_train,Y_train,seed, batch_size), 
                              steps_per_epoch = np.ceil(float(n_train_samples)/float(batch_size)),
                              validation_data = val_datagen.flow(X_val,Y_val,batch_size), 
                              validation_steps = np.ceil(float(n_val_samples)/float(batch_size)),
                              shuffle = True,
                              epochs = n_epochs,
                              callbacks = callbacks_list)

* Salvataggio del modello allenato

In [ ]:
model_path = current_dir + '/trained_net.h5'
model.save(model_path)